# Data exploration

Sam Maurer, May 2018 | Python 3.6

This notebook is for development and testing of the base data tables being used for the lab model. It also demonstrates ways to explore data registered with Orca.